# Talks markdown generator for academicpages

Takes a TSV of talks with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `talks.py`. Run either from the `markdown_generator` folder after replacing `talks.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases, rather than Stuart's non-standard TSV format and citation style.

In [9]:
import pandas as pd
import os

## Data format

The TSV needs to have the following columns: title, type, url_slug, venue, date, location, talk_url, description, with a header at the top. Many of these fields can be blank, but the columns must be in the TSV.

- Fields that cannot be blank: `title`, `url_slug`, `date`. All else can be blank. `type` defaults to "Talk" 
- `date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. 
    - The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/talks/YYYY-MM-DD-[url_slug]`
    - The combination of `url_slug` and `date` must be unique, as it will be the basis for your filenames

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [10]:
!cat talks.tsv


Multimodal Conversational AI Agents Towards Smarter Assistant	Invited Talk		Microsoft Research	07/02/2024	"Cambridge, United Kingdom"		"Artificial Intelligence (AI) is playing an increasingly vital role in scientific research, particularly in enhancing Human-Computer Interaction (HCI) through the development of smarter agents. Conversational AI agents, in particular, have demonstrated considerable potential in enriching the user experience within multimodal immersive systems, facilitated by large language model (LLM)-powered autonomous agents. 

In this talk, drawing from my practical expertise, we will delve into the application of large foundation models and frameworks in natural language processing (NLP) and information retrieval (IR) tasks. Specifically, we will explore recent advancements in multimodal agents within Extended Reality (XR) environments utilizing LLMs such as OpenAI's GPT models, LLAMA2, etc. We will also discuss model fine-tuning techniques like LORA and QLORA, as 

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [22]:
talks = pd.read_csv("talks.tsv", sep="\t", header=0)
talks

,title,type,url_slug,venue,date,location,talk_url,description
0,Multimodal Conversational AI Agents Towards Sm...,Invited Talk,msr2024,Microsoft Research,07/02/2024,"Cambridge, United Kingdom",NaN,Artificial Intelligence (AI) is playing an inc...
1,Multimodal Dialogue Agents Towards Intelligent XR,Invited Talk,uu2023,Utrecht University,04/12/2023,"Utrecht, Netherlans",NaN,Interactive personal assistants have been play...
2,Maximising efficiency in NLP model training an...,Invited Panel Talk,itw2023,Immersive Tech Week,29/11/2023,"Rotterdam, Netherlands",NaN,In the realm where Natural Language Processing...
3,Language Model Powered Dialogue Agents and Vir...,Invited Talk,sony2023,Sony AI,19/09/2023,"Barcelona, Spain",NaN,"In this talk, we will explore the emerging int..."
4,Advancements and Prospects in Dialogue Agents ...,Invited Talk,bosch2023,Bosch Center of Artificial Intelligence (BCAI),24/07/2023,"Renningen, Germany",NaN,"In this talk, we will delve into the exciting ..."
5,Generative AI towards Scientific Discovery,Invited Talk,msr2023,Microsoft Research AI4Science,06/04/2023,"Amsterdam, Netherlands",NaN,Artificial Intelligence (AI) is becoming incre...
6,Learning Embeddings to Represent Information R...,Invited Conference Talk,amlc2022,Amazon Machine Learning Conference (AMLC),14/10/2022,"Seattle, WA, US",NaN,"In this talk, I introduce our paper accepted b..."
7,Frontiers of Collaborative Task-oriented Dialo...,Invited Talk,ucl2022,University College London (UCL),06/07/2022,"London, UK",NaN,"In this talk, I introduce our exploration of c..."
8,Transformer Uncertainty Estimation with Hierar...,Invited Talk,sea2022,Search Engine Amsterdam (SEA),29/04/2022,"Amsterdam, Netherlands",NaN,"In this talk, I introduce the stochastic trans..."


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [23]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    if type(text) is str:
        return "".join(html_escape_table.get(c,c) for c in text)
    else:
        return "False"

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [24]:
loc_dict = {}

for row, item in talks.iterrows():
    
    md_filename = str(item.date) + "-" + item.url_slug + ".md"
    html_filename = str(item.date) + "-" + item.url_slug 
    year = item.date[:4]
    
    md = "---\ntitle: \""   + item.title + '"\n'
    md += "collection: talks" + "\n"
    
    if len(str(item.type)) > 3:
        md += 'type: "' + item.type + '"\n'
    else:
        md += 'type: "Talk"\n'
    
    md += "permalink: /talks/" + html_filename + "\n"
    
    if len(str(item.venue)) > 3:
        md += 'venue: "' + item.venue + '"\n'
        
    if len(str(item.location)) > 3:
        md += "date: " + str(item.date) + "\n"
    
    if len(str(item.location)) > 3:
        md += 'location: "' + str(item.location) + '"\n'
           
    md += "---\n"
    
    
    if len(str(item.talk_url)) > 3:
        md += "\n[More information here](" + item.talk_url + ")\n" 
        
    
    if len(str(item.description)) > 3:
        md += "\n" + html_escape(item.description) + "\n"
        
        
    md_filename = os.path.basename(md_filename)
    #print(md)
    
    with open("../_talks/" + md_filename, 'w') as f:
        f.write(md)

These files are in the talks directory, one directory below where we're working from.

In [25]:
!ls ../_talks

2022-amlc2022.md  2023-bosch2023.md 2023-sony2023.md
2022-sea2022.md   2023-itw2023.md   2023-uu2023.md
2022-ucl2022.md   2023-msr2023.md   2024-msr2024.md
